In [ ]:
! pip install azure-search-documents --pre
! pip install python-dotenv

In [1]:
# Import required libraries  
import os  
import json  
import openai
import keys
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)

import pandas as pd
  
# Configure environment variables  
load_dotenv()  
service_endpoint = "https://aimlexplorationsearch.search.windows.net"
index_name = "rsaf-cognitive-search"
key = "hwioAbEseT8hs3Pv98fREcnMpoqPQnp9FdR0OwoPpmAzSeCG2vMW"

# key = "wGvVdInupyqUH2YraI1mpZmviDLXjh3v3i2zZetKAnAzSeCZGrhS"
# service_endpoint = "https://rsaf-cognitive-search-service-us.search.windows.net"

openai.api_type = "azure"
openai.api_key = "0e576c031e5446e19cc6d866ad3d9f3f"
openai.api_base = "https://raid-ses-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
credential = AzureKeyCredential(key)

In [2]:
client_index = SearchIndexClient(endpoint=service_endpoint, credential=credential)

In [36]:
client_index.delete_index("rsaf-cognitive-search")

In [3]:
for i in client_index.list_indexes():
    print(i.name)

rsaf-cognitive-search


In [83]:
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryLanguage,
    QueryType  
)

search_client = SearchClient(endpoint=service_endpoint, index_name="rsaf-cognitive-search", credential=credential)

all_docs = search_client.search(search_text="fenestron", query_type=QueryType.SEMANTIC, query_language=QueryLanguage.EN_US, semantic_configuration_name='default',top=6)

pd.DataFrame(all_docs)

,li_jsonMetadata,id,content,total_pages,filename,page,hyperlink,embedding,li_doc_id,@search.score,@search.reranker_score,@search.highlights,@search.captions
0,"{""total_pages"": 50, ""page"": 13, ""filename"": ""E...",1be1e6da-488b-45be-973d-62952dfbcc5a,OFFICIAL (CLOSED)\nANNEX A\nTO EEM CHAPTER 1\n...,50,EEM FINAL caa201221-1-50.pdf,13,google.com,"[-0.0132496674, -0.0136131411, -0.0230840482, ...",5682c550-a5dd-4990-8bed-494e6cc20b02,2.511684,2.744638,None,None
1,"{""total_pages"": 50, ""page"": 15, ""filename"": ""E...",2ac5e300-d404-4b13-bde5-6d723a3cb7da,The fenestron is different in that the aerofoi...,50,EEM FINAL caa201221-1-50.pdf,15,google.com,"[-0.01364111, 0.00528065627, -0.0216288939, -0...",8cd5be3c-440d-486b-8f86-3593200b4d87,3.053105,2.561581,None,None
2,"{""total_pages"": 50, ""page"": 14, ""filename"": ""E...",b8cf4a55-c1c7-4e66-81d0-da2eda09fd81,OFFICIAL (CLOSED)\nhas seven stator vanes that...,50,EEM FINAL caa201221-1-50.pdf,14,google.com,"[-0.0137834195, 0.004918159, -0.0193652511, -0...",b25e03ba-f14a-414c-a3cd-84499d7320af,1.766508,2.349035,None,None
3,"{""total_pages"": 50, ""page"": 23, ""filename"": ""E...",3e0121c4-2b18-4bee-8bca-92560376a556,OFFICIAL (CLOSED)\noccurs when the collective ...,50,EEM FINAL caa201221-1-50.pdf,23,google.com,"[-0.0246961173, -0.0129821664, 0.00747292768, ...",0bc311b8-3aba-46c7-bb96-babcf0071112,1.362892,2.157839,None,None


In [4]:
search_client = SearchClient(endpoint=service_endpoint, index_name="rsaf-cognitive-search", credential=credential)

all_docs = search_client.search(search_text="*", top=50)

In [77]:
for i in all_docs:
    text = "Filename: " + i["filename"] + "\n" + "Hyperlink: " + i["hyperlink"] + "\n" + "Page Number: " + str(i["page"]) + "\n" + "Content: " + i["content"] + "\n-----------------------"

In [6]:
doc = search_client.search(search_text="*", filter="page eq 15")

pd.DataFrame(doc)

,hyperlink,li_doc_id,li_jsonMetadata,total_pages,content,page,id,filename,embedding,@search.score,@search.reranker_score,@search.highlights,@search.captions
0,google.com,df43746f-f64b-4e33-8d65-915d49731643,"{""total_pages"": 319, ""page"": 15, ""filename"": ""...",319,OFFICIAL (CLOSED)\n1A.4 Horizontal Stabilizer....,15,7a6afeb2-8f60-4a62-aff1-5e092cb0c0aa,EEM FINAL caa201221.pdf,"[-0.01948246, 0.0110256383, 0.0025789584, -0.0...",1.0,None,None,None
1,google.com,df43746f-f64b-4e33-8d65-915d49731643,"{""total_pages"": 319, ""page"": 15, ""filename"": ""...",319,The fenestron is different in that the aerofoi...,15,246e7eaa-04bc-4e36-8b52-0feeb7e4a37a,EEM FINAL caa201221.pdf,"[-0.009689129, 0.0116003519, -0.01604587, -0.0...",1.0,None,None,None


In [20]:
DOCUMENT = [{"id" : "2ac5e300-d404-4b13-bde5-6d723a3cb7da"}]

search_client.delete_documents(documents=DOCUMENT)

### Vector Search

In [12]:
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="swiftfaq-ada002")
    embeddings = response['data'][0]['embedding']
    return embeddings

In [30]:
search_client = SearchClient(endpoint=service_endpoint, index_name="chat-demo", credential=credential)

pd.DataFrame(search_client.search("autorotation", top=3))

,embedding,id,li_jsonMetadata,li_doc_id,page_label,content,@search.score,@search.reranker_score,@search.highlights,@search.captions
0,"[-0.021303589, -0.008744429, -0.020742599, -0....",21c6d4c3-6106-4f3d-b628-73e129507b7c,"{""page_label"": ""111"", ""file_name"": ""EEM FINAL ...",2da13a2e-fbc3-4d15-b8fa-6e1517074460,111,OFFICIAL (CLOSED) \n11-1 \nOFFICIAL (CLOSED)...,3.408726,None,None,None
1,"[-0.017312882, -0.0033271702, -0.012544677, -0...",2f71f668-85dc-4155-9b57-104767b0c28b,"{""page_label"": ""42"", ""file_name"": ""EEM FINAL c...",87546a10-0794-45fa-a8cf-46919ebe14ce,42,OFFICIAL (CLOSED) \n5-1 \nOFFICIAL (CLOSED) ...,3.335187,None,None,None
2,"[-0.022873871, -0.013425363, -0.0023482218, -0...",2282ddc7-7b38-4f25-815e-dc8d3f323e1d,"{""page_label"": ""116"", ""file_name"": ""EEM FINAL ...",bd14dd00-8091-4885-9421-86003faa860b,116,OFFICIAL (CLOSED) \n11-6 \nOFFICIAL (CLOSED)...,3.253138,None,None,None
